# Modeling
In this notebook, we'll be modeling the data we've previously prepared. Out notebook will be laid out as follows:

1. Model Selection & Generation
2. Hyperparameter Optimization
3. Fine-Tuning (if needed)
4. Reporting Best Model(s) + Settings
5. Interpretation
6. Conclusion

Our eventual goal here is two-fold:

1. Accurately [and fairly] model the diabetes dataset
2. Interpret the results to find something worth recommending to those wanting to reduce risk of diabetes. This can be via LIME/SHAP (i.e. some interpretable model that approximates the neural network) or via analyzing a more simple model's structure (i.e. regression coefficients, random forest decision boundaries)

In [2]:
# Environment Setup
from utils.model import *
from utils.dataset import *

***
## Model Selection & Generation

In [3]:
# generate lookup for models
models = {
    # "tree": TreeClassifier(target="diabetes", path="../datasets/pre_split_processed.parquet"),
    "ffnn": MLPClassifier(target="diabetes", path="../datasets/pre_split_processed.parquet")
}

# manual search
models["ffnn"].set_hyperparams({
    "learning_rate": .0005,
    "batch_size": 512,
    "num_hidden": 2,
    "hidden_size": 2048,
    "num_epochs": 50,
    "classify_fn": "sigmoid"
})

# train & test basic model
for mt, model in models.items():
    # attempt to load, else train and test
    # if not model.load_model():
    model.train_model(verbose=1)
    model.test_model()

<Train-Test Split Report>
Train: 512886 obs, 170962 no diabetes [0], 170962 pre-diabetes [1], 170962 diabetes [2]
Test: 50736 obs, 42741 no diabetes [0], 926 pre-diabetes [1], 7069 diabetes [2]

<Test Report>
Precision: [no diabetes] 0.8560014750169013, [pre-diabetes] 0.07317073170731707, [diabetes] 0.47493887530562345
Recall: [no diabetes] 0.9776093212606163, [pre-diabetes] 0.02267818574514039, [diabetes] 0.10991653699250248
F1-Score: [no diabetes] 0.9127727898289534, [pre-diabetes] 0.03462489694971146, [diabetes] 0.1785180930499713
Support: [no diabetes] 42741, [pre-diabetes] 926, [diabetes] 7069
Accuracy: 83.9286%


***
## Hyperparameter Optimization

In [3]:
# optimize hyperparams
optimizer_results = {model_type: model.optimize_hyperparams(kfold=2) for model_type, model in models.items()}
print(optimizer_results)

<Grid-Search>
Testing 216 combinations WITHOUT cross-validation
<testing> hidden_size=64, lr=0.001, bs=128, num_hidden=16, num_epochs=25. . . 

100%|██████████| 4007/4007 [00:13<00:00, 292.66it/s]



<Test Report>
Precision: [no diabetes] 0.8882522397036943, [pre-diabetes] 0.02496826068556919, [diabetes] 0.22292254105158402
Recall: [no diabetes] 0.6564890854214922, [pre-diabetes] 0.19114470842332612, [diabetes] 0.3802518036497383
F1-Score: [no diabetes] 0.7549845284541907, [pre-diabetes] 0.04416718652526513, [diabetes] 0.28106864641606105
Support: [no diabetes] 42741, [pre-diabetes] 926, [diabetes] 7069
Accuracy: 60.9508%
perf: 0.6095
<testing> hidden_size=64, lr=0.001, bs=256, num_hidden=16, num_epochs=25. . . 

100%|██████████| 2004/2004 [00:08<00:00, 246.12it/s]



<Test Report>
Precision: [no diabetes] 0.8864771221848569, [pre-diabetes] 0.02254923493667179, [diabetes] 0.236318407960199
Recall: [no diabetes] 0.6105846844949814, [pre-diabetes] 0.3326133909287257, [diabetes] 0.25534021785259586
F1-Score: [no diabetes] 0.7231088944305902, [pre-diabetes] 0.04223517312307165, [diabetes] 0.24546134493778474
Support: [no diabetes] 42741, [pre-diabetes] 926, [diabetes] 7069
Accuracy: 55.6015%
perf: 0.5560
<testing> hidden_size=128, lr=0.001, bs=128, num_hidden=8, num_epochs=25. . . 

...


<Test Report>
Precision: [no diabetes] 0.8560749774164408, [pre-diabetes] 0.024390243902439025, [diabetes] 0.22825008269930533
Recall: [no diabetes] 0.8869001661168433, [pre-diabetes] 0.01079913606911447, [diabetes] 0.19521855990946385
F1-Score: [no diabetes] 0.8712149940818883, [pre-diabetes] 0.014970059880239521, [diabetes] 0.21044605413648493
Support: [no diabetes] 42741, [pre-diabetes] 926, [diabetes] 7069
Accuracy: 77.4539%
perf: 0.7745
<testing> hidden_size=2048, lr=0.0005, bs=128, num_hidden=2, num_epochs=25. . . 

100%|██████████| 4007/4007 [00:48<00:00, 83.25it/s]



<Test Report>
Precision: [no diabetes] 0.8579440097688933, [pre-diabetes] 0.022058823529411766, [diabetes] 0.23796265967900426
Recall: [no diabetes] 0.8876722584871669, [pre-diabetes] 0.009719222462203024, [diabetes] 0.20554533880322534
F1-Score: [no diabetes] 0.8725549946528983, [pre-diabetes] 0.013493253373313344, [diabetes] 0.22056925996204935
Support: [no diabetes] 42741, [pre-diabetes] 926, [diabetes] 7069
Accuracy: 77.6608%
perf: 0.7766
<testing> hidden_size=2048, lr=0.0005, bs=256, num_hidden=2, num_epochs=25. . . 

100%|██████████| 2004/2004 [00:32<00:00, 62.47it/s]



<Test Report>
Precision: [no diabetes] 0.8559900682805711, [pre-diabetes] 0.0213903743315508, [diabetes] 0.2411496003045299
Recall: [no diabetes] 0.9033948667555742, [pre-diabetes] 0.008639308855291577, [diabetes] 0.17923327203281936
F1-Score: [no diabetes] 0.8790538310054753, [pre-diabetes] 0.012307692307692308, [diabetes] 0.20563174551651384
Support: [no diabetes] 42741, [pre-diabetes] 926, [diabetes] 7069
Accuracy: 78.6168%
perf: 0.7862
{
    "learning_rate": 0.0005,
    "input_size": 21,
    "output_size": 3,
    "hidden_size": 2048,
    "num_hidden": 2,
    "num_epochs": 25,
    "batch_size": 256,
    "classify_fn": "sigmoid"
}

<Test Report>
Precision: [no diabetes] 0.8559900682805711, [pre-diabetes] 0.0213903743315508, [diabetes] 0.2411496003045299
Recall: [no diabetes] 0.9033948667555742, [pre-diabetes] 0.008639308855291577, [diabetes] 0.17923327203281936
F1-Score: [no diabetes] 0.8790538310054753, [pre-diabetes] 0.012307692307692308, [diabetes] 0.20563174551651384
Support: [n

***
## Fine-Tuning + Other Adjustments

***
## Best Model Report

***
## Interpretation

***
## Conclusion